In [1]:
# Packages for analysis
import pandas as pd
import numpy as np
from sklearn import svm, datasets
from sklearn.metrics import matthews_corrcoef

# Packages for visuals
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(font_scale=1.2)

# Allows charts to appear in the notebook
%matplotlib inline

# Pickle package
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
import subprocess
import sys
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neural_network import MLPClassifier

In [2]:
#peptides = pd.read_csv('train.csv')
peptides = pd.read_csv('peptides-all75.csv', encoding='cp1252')
peptides

,Name,Type,Seq,SeqL,MwWt,Aromaticity,isoelectric point,A,R,N,...,_HydrophobicityD2001,_HydrophobicityD2025,_HydrophobicityD2050,_HydrophobicityD2075,_HydrophobicityD2100,_HydrophobicityD3001,_HydrophobicityD3025,_HydrophobicityD3050,_HydrophobicityD3075,_HydrophobicityD3100
0,sp|P40505|61-130,0,QYARQVRDLEEERDLELVRLRLFEEYRVSRSGIEFQEDIEKAKAEH...,70,8610.7265,0.071429,5.957659,4.286,11.429,0.000,...,2.857,37.143,45.714,65.714,85.714,8.571,25.714,47.143,74.286,97.143
1,sp|P40505|131-135,0,ERLLM,5,660.8262,0.000000,6.101802,0.000,20.000,0.000,...,0.000,0.000,0.000,0.000,0.000,60.000,100.000,60.000,80.000,100.000
2,sp|Q6GIL6|1-70,0,MRTPIIAGNWKMNKTVQEAKDFVNALPTLPDSKEVESVICAPAIQL...,70,7576.6359,0.057143,5.179817,12.857,1.429,5.714,...,4.286,27.143,58.571,74.286,98.571,1.429,17.143,41.429,62.857,100.000
3,sp|Q6GIL6|71-140,0,EDNGAFTGETSPVALADLGVKYVVIGHSERRELFHETDEEINKKAH...,70,7740.4796,0.057143,4.916652,7.143,4.286,2.857,...,5.714,15.714,38.571,72.857,98.571,8.571,25.714,47.143,70.000,84.286
4,sp|Q6GIL6|141-210,0,ANDVVGEQVKKAVAGLSEDQLKSVVIAYEPIWAIGTGKSSTSEDAN...,70,7443.2487,0.042857,4.575789,12.857,2.857,2.857,...,1.429,24.286,51.429,71.429,97.143,5.714,22.857,44.286,72.857,100.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2657,PA-MAP,1,LAAKLTKAATKLTAALTKLAAALT,24,2354.8707,0.000000,10.477740,37.500,0.000,0.000,...,8.333,25.000,54.167,70.833,100.000,4.167,4.167,50.000,66.667,95.833
2658,HSAFP1,1,DGVKLCDVPSGTWSGHCGSSSKCSQQCKDREHFAYGGACHYQFPSV...,54,5948.6696,0.111111,8.490825,3.704,3.704,0.000,...,3.704,22.222,37.037,66.667,83.333,5.556,14.815,50.000,85.185,100.000
2659,HSLIN06,1,EHFAYGGAKHYQFPSVKKFKKRQK,24,2910.3355,0.208333,10.218835,8.333,4.167,0.000,...,8.333,16.667,29.167,41.667,62.500,12.500,12.500,54.167,66.667,79.167
2660,Verine,1,RRRWWWWV,8,1330.5430,0.500000,11.999968,0.000,37.500,0.000,...,0.000,0.000,0.000,0.000,0.000,50.000,50.000,62.500,75.000,100.000


In [3]:
characters=peptides.to_numpy()
#type_label = np.where(peptides['Type'] =='Positive', 1, 0)
type_label=peptides['Type'].to_numpy()

In [4]:
X=characters
Y=type_label

In [5]:
X

array([['sp|P40505|61-130', 0,
        'QYARQVRDLEEERDLELVRLRLFEEYRVSRSGIEFQEDIEKAKAEHEKLIKLCKERLYSSIEQKIKKLQE',
        ..., 47.143, 74.286, 97.14299999999999],
       ['sp|P40505|131-135', 0, 'ERLLM', ..., 60.0, 80.0, 100.0],
       ['sp|Q6GIL6|1-70', 0,
        'MRTPIIAGNWKMNKTVQEAKDFVNALPTLPDSKEVESVICAPAIQLDALTTAVKEGKAQGLEIGAQNTYF',
        ..., 41.428999999999995, 62.857, 100.0],
       ...,
       ['HSLIN06', 1, 'EHFAYGGAKHYQFPSVKKFKKRQK', ..., 54.167, 66.667,
        79.167],
       ['Verine', 1, 'RRRWWWWV', ..., 62.5, 75.0, 100.0],
       ['Phylloseptin-PTa', 1, 'FLSLIPKIAGGIAALAKHL', ..., 26.316,
        63.158, 100.0]], dtype=object)

In [6]:
Y

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

##### Method for caling Motif script and counting the frequencies of motif

In [7]:
def call_and_parse_motif_on_train (x_data) :
    if os.path.exists("pos_train_seq_new.fasta"):
        os.remove("pos_train_seq_new.fasta")
    if os.path.exists("neg_train_seq_new.fasta"):
        os.remove("neg_train_seq_new.fasta")
    if os.path.exists("motif_output.occurrences"):
        os.remove("motif_output.occurrences")
        #os.rename('C:\\Users\\bipas\motif_output_Sept14.occurrences', 
        #          'C:\\Users\\bipas\motif_output_Sept14_'+str(n)+'.occurrences')
    fp= open("pos_train_seq_new.fasta","a+")
    fn= open("neg_train_seq_new.fasta","a+")
    for rows in x_data:
        if(rows.item(1) == 1):
            fp.write('>')
            fp.write(str(rows.item(0)))
            fp.write('\n')
            fp.write((str(rows.item(2))) + '\n')
            #fp.write('\n')
    for rows in x_data:
        if(rows.item(1) == 0):
            fn.write('>')
            fn.write(rows.item(0))
            fn.write('\n')
            fn.write((str(rows.item(2)) )+ '\n')
            #fn.write('\n') 
    fp.close()  
    fn.close()
    #print("running motif script")
    pl_script = subprocess.Popen(['C:\Strawberry\perl\\bin\perl.exe', 
                                  'C:\\Users\\bipas\MERCI\MERCI-U.pl', 
                                  '-p', "C:\\Users\\bipas\\pos_train_seq_new.fasta", 
                                  '-n', "C:\\Users\\bipas\\neg_train_seq_new.fasta",  
                                  '-o', "motif_output",
                                  #'-c', "BETTS-RUSSELL",
                                  '-k', "ALL"
                                 ])
                                  #'-c', "BETTS-RUSSELL"])
    pl_script.communicate()
    #print("motif script end, now parsing")
    occ_file='C:\\Users\\bipas\motif_output.occurrences'
    f = open(occ_file, "r")
    occ_word_list = f.read().split()
    f.close()

    #print (x_data.shape)
    Xm=x_data[:,3:]
    #print (Xm.shape)
    N=x_data[:,0].shape
    b=np.zeros((N[0],1))
    Xm = np.hstack((b, Xm))
    i=0
    max = 0
    for x in x_data:
        name='>'+x[0]
        #print(x[0])
        m=occ_word_list.count(name)
        #print(m)
        Xm[i,0]=m
        #if (m>max):
        #    max = m
        i=i+1
    return Xm
    

In [8]:
def call_and_parse_motif_on_test (x_data) :
    if os.path.exists("pos_test_seq.fasta"):
        os.remove("pos_test_seq.fasta")
    if os.path.exists("neg_test_seq.fasta"):
        os.remove("neg_test_seq.fasta")
    if os.path.exists("motif_test_output"):
        os.remove("motif_test_output")
    fp= open("pos_test_seq.fasta","a+")
    fn= open("neg_test_seq.fasta","a+")
    for rows in x_data:
        if(rows.item(1) == 1):
            fp.write('>')
            fp.write(str(rows.item(0)))
            fp.write('\n')
            fp.write((str(rows.item(2))) + '\n')
            #fp.write('\n')
    for rows in x_data:
        if(rows.item(1) == 0):
            fn.write('>')
            fn.write(rows.item(0))
            fn.write('\n')
            fn.write((str(rows.item(2)) )+ '\n')
            #fn.write('\n') 
    fp.close()  
    fn.close()
    #print("running motif script")
    pl_script = subprocess.Popen(['C:\Strawberry\perl\\bin\perl.exe', 
                                  'C:\\Users\\bipas\MERCI\MERCI_motif_locator.pl', 
                                  '-p', "C:\\Users\\bipas\\pos_test_seq.fasta", 
                                  '-n', "C:\\Users\\bipas\\neg_test_seq.fasta",  
                                  '-i', "motif_output",
                                 # '-c', "BETTS-RUSSELL",
                                  '-o', "motif_test_output"])
                                 # '-c', "BETTS-RUSSELL"])
    pl_script.communicate()
    #print("motif script end, now parsing")
    occ_file='C:\\Users\\bipas\motif_test_output'
    f = open(occ_file, "r")
    occ_word_list = f.read().split()
    f.close()

    #print (x_data.shape)
    Xm=x_data[:,3:]
    #print (Xm.shape)
    N=x_data[:,0].shape
    b=np.zeros((N[0],1))
    Xm = np.hstack((b, Xm))
    i=0
    max=0
    for x in x_data:
        name='>'+x[0]
        #print(x[0])
        m=occ_word_list.count(name)
        #print(m)
        Xm[i,0]=m
        #if (m>max):
        #    max = m
        i=i+1
    return Xm
    

##### MinMax Scalar

In [9]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))

###### Divide into test and train set

In [10]:
X_tr, X_te, y_tr, y_te = train_test_split(X, Y, stratify=Y,  test_size=0.2, random_state=42, shuffle=True)

In [11]:
X_te.shape

(533, 574)

In [12]:
y_te.shape

(533,)

##### SVM stratitified sampling

In [13]:
#SS_classifier = svm.SVC(kernel='rbf', C=150, gamma=0.05, probability=True)
SS_classifier = svm.SVC(kernel='rbf', C=150, gamma=0.05)
sss = StratifiedKFold(n_splits =10, random_state=42, shuffle=True)
#sss.get_n_splits(peptides)

sum_SS_f1=0
scores = []
mccs = []
f1s = []
n=0
#sss = StratifiedShuffleSplit(n_splits= 10, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(X_tr, y_tr):
    positive=0
    negative=0
    for i in range(y_tr.shape[0]):
        if (y_tr[i]):
            positive=positive+1
        else:
            negative=negative+1
    print('positive sample', positive)
    print('negative sample', negative)
    X_SS_train, X_SS_test, y_SS_train, y_SS_test = X_tr[train_index], X_tr[test_index], y_tr[train_index], y_tr[test_index]
    
    X_SS_train_new=call_and_parse_motif_on_train(X_SS_train)
    X_SS_train_new = min_max_scaler.fit_transform(X_SS_train_new)
    
    X_SS_test_new=call_and_parse_motif_on_test(X_SS_test)
    #X_SS_test_new = normalize (X_SS_test_new, max_c, min_c)
    X_SS_test_new = min_max_scaler.transform(X_SS_test_new)
    #print("test size after motif adding" , X_SS_test_new.shape)
    n=n+1
    SS_classifier.fit(X_SS_train_new, y_SS_train)
    scores.append(SS_classifier.score(X_SS_test_new, y_SS_test))
    ypred=(SS_classifier.predict(X_SS_test_new))
    mcc=matthews_corrcoef(y_SS_test, ypred)
    print("MCC", mcc)
    mccs.append (mcc)
    #print("accuracy", (SS_classifier.score(X_SS_test_new, y_SS_test)))
    f1=f1_score(y_SS_test, ypred)
    print("F1", f1)
    f1s.append(f1)
    print("*************************************")
    
        
print("*************************************")
print("Scores: ",np.min(scores), np.max(scores), np.std(scores))
print("F1s: ", np.min(f1s), np.max(f1s), np.std(f1s))
print("MCCs: ", np.min(mccs), np.max(mccs), np.std(mccs))
print ( "avg cross-validation accuracy:", (sum(scores)/10))
print ( "avg cross-validation f1:", (sum(f1s)/10))
print ( "avg cross-validation mcc:", (sum(mccs)/10))
print("*************************************")
#SS_classifier.fit(best_X_SS_train, best_y_SS_train)
X_new = call_and_parse_motif_on_train(X_tr)
#X_new, max_c, min_c = normalize_and_get_minmax (X_new)
X_new = min_max_scaler.fit_transform(X_new)
SS_classifier.fit(X_new,y_tr)
y_tr_predict = SS_classifier.predict(X_new)

print("*************************************")

print('f1 on Train set: ', f1_score(y_tr, y_tr_predict))
print('MCC on Train set: ', matthews_corrcoef(y_tr, y_tr_predict))
tn, fp, fn, tp = confusion_matrix(y_tr, y_tr_predict).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn+fp)
print('Specificity on Train set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp+fn)
print('Sensitivity on Train set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp+tn) /(tp+tn+fp+fn)
print('Accuracy on Train set: ', accuracy)

X_te_new = call_and_parse_motif_on_test(X_te)
x_test_df = pd.DataFrame(X_te_new)
x_test_df.to_csv('x_test1.csv')
X_te_new = min_max_scaler.transform(X_te_new)
x_test_df = pd.DataFrame(X_te_new)
x_test_df.to_csv('x_test2.csv')
y_SS_pred=SS_classifier.predict(X_te_new)


#TESTING NEW CODE
#for i in range(X_te_new.shape[0]):
#    if (y_te[i]):
#        if (y_SS_pred[i] != y_te[i]):
           # print ("Wrong prediction for object: ", i)
#            results = SS_classifier.predict_proba(X_te_new)[i]
           # print ("Actual: ", y_te[i])
           # print ("Predtiction: ", y_SS_pred[i])
            # gets a dictionary of {'class_name': probability}
#            prob_per_class_dictionary = dict(zip(SS_classifier.classes_, results))
           # print (prob_per_class_dictionary)

#for i in range(X_te_new.shape[0]):
#    if (y_te[i]):
#        if (y_SS_pred[i] == y_te[i]):
#            print ("Correct prediction for object: ", i)
#            results = SS_classifier.predict_proba(X_te_new)[i]
#            print ("Actual: ", y_te[i])
#            print ("Predtiction: ", y_SS_pred[i])
#            # gets a dictionary of {'class_name': probability}
#            prob_per_class_dictionary = dict(zip(SS_classifier.classes_, results))
#            print (prob_per_class_dictionary)
#TESTING NEW CODE


print("*************************************")
print('f1 on Test set: ', f1_score(y_te, y_SS_pred))
print('MCC on Test set: ', matthews_corrcoef(y_te, y_SS_pred))
tn, fp, fn, tp = confusion_matrix(y_te, y_SS_pred).ravel()
print("tn, fp, tp, fn", tn, fp, tp, fn)
specificity = tn / (tn+fp)
print('Specificity on Test set(tn / (tn+fp)): ', specificity)
sensitivity = tp / (tp+fn)
print('Sensitivity on Test set(tp / (tp+fn)): ', sensitivity)
accuracy = (tp+tn) /(tp+tn+fp+fn)
print('Accuracy on Test set: ', accuracy)

filename = 'finalized_model_without_probab.sav'
pickle.dump(SS_classifier, open(filename, 'wb'))

positive sample 194
negative sample 1935
MCC 0.781074803302132
F1 0.7878787878787878
*************************************
positive sample 194
negative sample 1935
MCC 0.847542615169233
F1 0.8484848484848484
*************************************
positive sample 194
negative sample 1935
MCC 0.781074803302132
F1 0.7878787878787878
*************************************
positive sample 194
negative sample 1935
MCC 0.8195100309119416
F1 0.8333333333333333
*************************************
positive sample 194
negative sample 1935
MCC 0.8266413456321215
F1 0.8421052631578947
*************************************
positive sample 194
negative sample 1935
MCC 0.8859385267630794
F1 0.8947368421052632
*************************************
positive sample 194
negative sample 1935
MCC 0.8853001441193066
F1 0.888888888888889
*************************************
positive sample 194
negative sample 1935
MCC 0.823950527860629
F1 0.8235294117647058
*************************************
positive samp

##### Unknown test set (prediction set), currently working with 20:200 data

In [42]:
test_peptides = pd.read_csv('DRAMS_Anticancer_Jan.csv')
test_peptides

,Name,Seq,SeqL,MwWt,Aromaticity,PI,A,R,N,D,...,_HydrophobicityD2001,_HydrophobicityD2025,_HydrophobicityD2050,_HydrophobicityD2075,_HydrophobicityD2100,_HydrophobicityD3001,_HydrophobicityD3025,_HydrophobicityD3050,_HydrophobicityD3075,_HydrophobicityD3100
0,DRAMP00795,GIPCGESCVFIPCITAAIGCSCKSKVCYRN,30,3123.7349,0.066667,8.333780,6.667,3.333,3.333,0.000,...,3.333,16.667,50.000,63.333,93.333,6.667,26.667,36.667,60.000,90.000
1,DRAMP00798,GIPCGESCVFIPCITGAIGCSCKSKVCYRN,30,3109.7083,0.066667,8.333780,3.333,3.333,3.333,0.000,...,3.333,16.667,50.000,63.333,93.333,6.667,26.667,36.667,60.000,90.000
2,DRAMP01064,AWKLFDDGV,9,1050.1639,0.222222,4.207870,11.111,0.000,0.000,22.222,...,11.111,88.889,11.111,11.111,88.889,22.222,22.222,44.444,55.556,100.000
3,DRAMP01607,GLFDIIKKIAESF,13,1480.7458,0.153846,6.069006,7.692,0.000,0.000,7.692,...,7.692,92.308,7.692,76.923,92.308,15.385,15.385,38.462,46.154,100.000
4,DRAMP01608,GLLDIVKKVVGAFGSL,16,1615.9536,0.062500,8.590944,6.250,0.000,0.000,6.250,...,6.250,6.250,68.750,75.000,93.750,12.500,18.750,37.500,62.500,100.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,DRAMP18147,LKCNKLVPLFYKTCPAGKNL,20,2250.7674,0.100000,9.595362,5.000,0.000,10.000,0.000,...,40.000,40.000,65.000,75.000,85.000,5.000,15.000,35.000,50.000,100.000
70,DRAMP18491,RWKIFKKIEKMGRNIRDGIVKAGPAIEVLGSAKAIGK,37,4079.9020,0.054054,10.708602,10.811,8.108,2.703,2.703,...,32.432,48.649,64.865,83.784,97.297,5.405,13.514,40.541,70.270,94.595
71,DRAMP18494,FIHHIIGGLFSAGKAIHRLIRRRRR,25,2981.5575,0.080000,11.999968,8.000,24.000,0.000,0.000,...,12.000,16.000,32.000,48.000,68.000,4.000,8.000,24.000,40.000,80.000
72,DRAMP03575,FKRIVQRIKDFLR,13,1719.0850,0.153846,11.723012,0.000,23.077,0.000,7.692,...,0.000,0.000,0.000,0.000,0.000,7.692,7.692,38.462,61.538,92.308


In [43]:
X_test=test_peptides.to_numpy()
X_test

array([['DRAMP00795', 'GIPCGESCVFIPCITAAIGCSCKSKVCYRN', 30, ..., 36.667,
        60.0, 90.0],
       ['DRAMP00798', 'GIPCGESCVFIPCITGAIGCSCKSKVCYRN', 30, ..., 36.667,
        60.0, 90.0],
       ['DRAMP01064', 'AWKLFDDGV', 9, ..., 44.443999999999996,
        55.556000000000004, 100.0],
       ...,
       ['DRAMP18494', 'FIHHIIGGLFSAGKAIHRLIRRRRR', 25, ..., 24.0, 40.0,
        80.0],
       ['DRAMP03575', 'FKRIVQRIKDFLR', 13, ..., 38.461999999999996,
        61.538000000000004, 92.30799999999999],
       ['DRAMP18658', 'FLPIVAKLLSGLL', 13, ..., 38.461999999999996,
        69.23100000000001, 100.0]], dtype=object)

##### Calling Merci_motif_locator for the test set

In [48]:
def call_and_parse_motif_on_test_data (x_data) :
    if os.path.exists("test_seq_new.fasta"):
        os.remove("test_seq_new.fasta")
    if os.path.exists("motif_test_output_data"):
        os.remove("motif_test_output_data")
    ft= open("test_seq_new.fasta","a+")
    for rows in x_data:
        ft.write('>')
        ft.write(str(rows.item(0)))
        ft.write('\n')
        ft.write((str(rows.item(1))) + '\n')
            #fp.write('\n')
    ft.close()  
    #fn.close()
    print("running motif script")
    pl_script = subprocess.Popen(['C:\Strawberry\perl\\bin\perl.exe', 
                                  'C:\\Users\\bipas\MERCI\MERCI_motif_locator.pl', 
                                  '-p', "C:\\Users\\bipas\\test_seq_new.fasta", 
                                  #'-n', "C:\\Users\\bipas\\test_seq.fasta",  
                                 # '-i', "motif_output_Jan",
                                  '-i', "motif_output",
                                  '-o', "motif_test_output_data"])
                                # '-c', "KOOLMAN-ROHM"])
    pl_script.communicate()
    print("motif script end, now parsing")
    occ_file='C:\\Users\\bipas\motif_test_output_data'
    f = open(occ_file, "r")
    occ_word_list = f.read().split()
    f.close()

    #print (x_data.shape)
    Xm=x_data[:,2:]
    print (Xm.shape)
    N=x_data[:,0].shape
    b=np.zeros((N[0],1))
    Xm = np.hstack((b, Xm))
    i=0
    max=0
    for x in x_data:
        name='>'+x[0]
        #print(x[0])
        m=occ_word_list.count(name)
        #print(m)
        Xm[i,0]=m
       
    return Xm
        

In [49]:
X_test.shape
X_test_new= call_and_parse_motif_on_test_data(X_test)

X_test_new.shape

running motif script
motif script end, now parsing
(74, 571)


(74, 572)

##### scaling the test dataset

In [50]:
X_test_new1 = min_max_scaler.transform(X_test_new)
X_test_new1

array([[4.57247371e-04, 3.66197183e-01, 3.14524998e-01, ...,
        3.66670000e-01, 6.00000000e-01, 9.00000000e-01],
       [0.00000000e+00, 3.66197183e-01, 3.12869708e-01, ...,
        3.66670000e-01, 6.00000000e-01, 9.00000000e-01],
       [0.00000000e+00, 7.04225352e-02, 6.98212485e-02, ...,
        4.44440000e-01, 5.55560000e-01, 1.00000000e+00],
       ...,
       [0.00000000e+00, 2.95774648e-01, 2.97746531e-01, ...,
        2.40000000e-01, 4.00000000e-01, 8.00000000e-01],
       [0.00000000e+00, 1.26760563e-01, 1.48761155e-01, ...,
        3.84620000e-01, 6.15380000e-01, 9.23080000e-01],
       [0.00000000e+00, 1.26760563e-01, 1.09189152e-01, ...,
        3.84620000e-01, 6.92310000e-01, 1.00000000e+00]])

###### loading the saved model

In [51]:
loaded_model = pickle.load(open('finalized_model_without_probab.sav', 'rb'))

###### calling prediction on test data set

In [52]:
y_test_pred=loaded_model.predict(X_test_new1)
#y_test_pred = (SS_classifier.predict_proba(X_test_new1)[:,1] >= 0.98).astype(int)

In [53]:
y_test_pred

array([1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1], dtype=int64)

In [23]:
#y_original_test= np.where(test_peptides['Type'] =='Positive', 1, 0)

In [54]:
def Sort_Tuple(tup):  
    tup.sort(reverse=True, key = lambda x: x[1])  
    return tup  

###### making an array of decision function and name

In [55]:
x_predict = []
for i in range(X_test.shape[0] ):
#for i in range(20):
    #if (y_original_test[i]):
        if (y_test_pred[i] == 1):
            print ("Prediction for object: ", X_test[i][0])
            results = loaded_model.decision_function(X_test_new1)[i]
            #print ("Actual: ", y_original_test[i])
            #print ("Predtiction: ", y_test_pred[i])
            # gets a dictionary of {'class_name': probability}
            #prob_per_class_dictionary = dict(zip(SS_classifier.classes_, results))
            print ('decision function: ', results)
            #print("prediction", y_test_pred[i])
            x_predict.append([X_test[i], results])
x_predict = Sort_Tuple(x_predict)
output = []
for i in x_predict:
    arr = np.array(i[0])
    output.append(np.append(arr, i[1]))

Prediction for object:  DRAMP00795
decision function:  0.09657298024685518
Prediction for object:  DRAMP00798
decision function:  0.08038013689631257
Prediction for object:  DRAMP01608
decision function:  0.10914870031266105
Prediction for object:  DRAMP01613
decision function:  0.39525952179935125
Prediction for object:  DRAMP01614
decision function:  0.22769110417529564
Prediction for object:  DRAMP01617
decision function:  0.09768118566640366
Prediction for object:  DRAMP01618
decision function:  0.27752324796008093
Prediction for object:  DRAMP01746
decision function:  0.6579401165515466
Prediction for object:  DRAMP03574
decision function:  0.9858598112694696
Prediction for object:  DRAMP03687
decision function:  0.6033094187680133
Prediction for object:  DRAMP02926
decision function:  1.0855275307245378
Prediction for object:  DRAMP03829
decision function:  1.225343568016876
Prediction for object:  DRAMP00771
decision function:  0.1800131779595286
Prediction for object:  DRAMP007

###### Inserting the decisoon into a CSV file

In [56]:
x_predict_df = pd.DataFrame(output)
x_predict_df.to_csv('prediction of model_cancer1.csv')

In [26]:
    occ_file='C:\\Users\\bipas\motif_test_output_data'
    f = open(occ_file, "r")
    occ_word_list = f.read().split()
    f.close()

In [27]:
i=0
print(output[1])
for row in output:
    name='>'+row[0]
    m=occ_word_list.count(name)
    my_array = np.empty(1)
    my_array[0]=m
    row=np.append(row, my_array, axis=0)
    re_arr = np.empty(row.shape, dtype=object)
    j=0
    for el in row:
        if (j>1):
            if (j>572):
                re_arr[j-573+2] = el
            else:
                re_arr[j+2]=el
        else:
            re_arr[j]=el
        j=j+1
    output[i]=re_arr
    i = i+1
print(output[1])

['DRAMP03575' 'FKRIVQRIKDFLR' 13 1719.085 0.153846154 11.72301159 0.0
 23.076999999999998 0.0 7.692 0.0 0.0 7.692 0.0 0.0 15.385 7.692 15.385
 0.0 15.385 0.0 0.0 0.0 0.0 0.0 7.692 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0
 0.0 0.0 0.0 0 0.0 0.0 0.0 0.0 0.0 0 0.0 0 0.0 0.0 0.0 0.0 0 0.0 0.0 0
 16.67 0.0 0.0 0 0.0 0.0 0 0 0.0 0 0.0 0 0.0 0.0 0 0 0.0 0 0 0 0.0 0.0 0.0
 0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0 0 0.0 0.0 0.0 0 0.0 0 0.0 0.0 0.0 0.0
 8.33 0.0 0.0 0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0 0.0 0 0.0 0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0 0.0 0.0 0.0 0 0 0 0.0 0.0 0 0.0 0 0 0.0 0.0 0 0.0 0 0.0
 0.0 0 0.0 0.0 0 8.33 0 0 0.0 0 0 0.0 0 0 0.0 0 0 0 0 0.0 0 0.0 0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0 0.0 0.0 0.0 0.0 0.0 0 0.0 0
 0.0 0.0 0 0 0 0 0 0.0 0.0 0 0 0 0 0.0 0 0 0 0 0 0.0 0.0 0.0 0.0 0.0 0.0 0
 0.0 0.0 0.0 0.0 8.33 0 0.0 0.0 0.0 0.0 0 0 8.33 0.0 8.33 0.0 0.0 0.0 0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0 0 0.0 0.0 8.33 0.0 8.33 0.0
 0.0 0 0.0 0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 

In [28]:

x_predict_df = pd.DataFrame(output)
x_predict_df.to_csv('prediction of model_cancer2.csv')